In [3]:
import pandas as pd
import os
import glob

os.listdir()

['Mexico_data_preprocessing.ipynb',
 'preprocessing.ipynb',
 'RSNA_data_preprocessing.ipynb',
 'test_set_rsna.ipynb']

In [4]:
os.chdir("../")

## Extracts information like actual age, estimated bone age, gender and radiologist initials from the text file.

In [3]:
import re


def calculate_total_months(duration):
    total_months = 0

    # Extract years and months from the duration string
    years = re.findall(r'(\d+)\s*y', duration, re.IGNORECASE)
    months = re.findall(r'(\d+)\s*m', duration, re.IGNORECASE)

    # Add the years and months to the total
    if years:
        total_months += int(years[0]) * 12
    if months:
        total_months += int(months[0])

    # If no years or months are found, check for a duration in months
    if not years and not months:
        months_only = re.findall(r'(\d+)\s*months?', duration, re.IGNORECASE)
        if months_only:
            total_months += int(months_only[0])

    return total_months


def check_gender(data):
    pattern = r'(?i)GENDER:\s*(M|F)'
    match = re.search(pattern, data)
    if match:
        return match.group(1).upper()
    else:
        return None


def clean_values(data):
    age = calculate_total_months(data[0])
    gender = [0 if data[1] == "MALE" else 1][0]

    # Check if the age is in the second line or third line
    if bool(re.search(r'\d', data[2])):
        bage = calculate_total_months(data[2])
        radiologist = data[3].split(":")[1].strip().upper()
    else:
        bage = calculate_total_months(data[3])
        radiologist = data[2].split(":")[1].strip().upper()

    return age, gender, bage, radiologist

# age, bage, gender, radiologist= [], [], [], []
# for dir in glob.glob("./data/Mex_sample_data/*"):
#     for file in glob.glob(dir + "/*.txt"):
#         with open(file, "r") as f:
#             data = f.readlines()
#             data = [line.strip() for line in data if line.strip()]
#
#             cleaned = clean_values(data)
#             age.append(cleaned[0])
#             gender.append(cleaned[1])
#             bage.append(cleaned[2])
#             radiologist.append(cleaned[3])
# mexico_data = pd.DataFrame(data=zip(age, gender, bage, radiologist), columns=['age', 'gender', 'bage', 'radiologist'])

In [4]:
age, bage, gender, radiologist, year_entry, id_, paths = [], [], [], [], [], [], []

In [32]:
years = [2019]
for year in years:
    for dir in glob.glob(f"data/Mexico_private_dataset/{year}/*"):
        for file in glob.glob(dir + "/*.txt"):
            with open(file, "r", encoding="utf8") as f:
                data = f.readlines()
                data = [line.strip() for line in data if line.strip()]
                id_.append(len(id_) + 1)
                dicom_file = glob.glob(dir + "/*.dcm")
                paths.append(dicom_file[0])
                cleaned = clean_values(data)
                age.append(cleaned[0])
                gender.append(cleaned[1])
                bage.append(cleaned[2])
                radiologist.append(cleaned[3])
                year_entry.append(year)


In [5]:
for year in [2023]:
    for dir in glob.glob(f"data/Mexico_private_dataset/{year}/*"):
        for file in glob.glob(dir + "/*.txt"):
            with open(file, "r", encoding="utf8") as f:
                data = f.readlines()
                data = [line.strip() for line in data if line.strip()]
                id_.append(len(id_) + 1)
                dicom_file = glob.glob(dir + "/*.dcm")
                paths.append(dicom_file[0])
                age.append(calculate_total_months(data[0]))
                if check_gender(data[1]) == "M":
                    gender.append(0)
                elif check_gender(data[1]) == "F":
                    gender.append(1)
                else:
                    gender.append(None)
                bage.append(calculate_total_months(data[2].split(":")[1]))
                radiologist.append(data[3].split(":")[1].strip().upper())
                year_entry.append(year)


In [11]:
mexico_data

,id,age,gender,boneage,radiologist,year_entry,path
0,1,84,0,72,JSA,2023,"data/Mexico_private_dataset/2023\AGAMI KALACH,..."
1,2,112,0,84,JSA,2023,"data/Mexico_private_dataset/2023\AGAMI KALACH,..."
2,3,63,1,36,JSA,2023,"data/Mexico_private_dataset/2023\AGAMI KALACH,..."
3,4,135,0,132,JSA,2023,"data/Mexico_private_dataset/2023\AGAMI KALACH,..."
4,5,154,1,144,JSA,2023,data/Mexico_private_dataset/2023\AGUILAR ALVAR...
...,...,...,...,...,...,...,...
111,112,97,1,84,JSA,2023,data/Mexico_private_dataset/2023\SAADIA DICKTE...
112,113,76,1,66,JSA,2023,data/Mexico_private_dataset/2023\SAADIA DICKTE...
113,114,138,1,144,JSA,2023,"data/Mexico_private_dataset/2023\SACAL ROMANO,..."
114,115,111,0,84,JSA,2023,data/Mexico_private_dataset/2023\SAINZ GONZALE...


In [6]:
mexico_data = pd.DataFrame(data=zip(id_, age, gender, bage, radiologist, year_entry, paths),
                           columns=['id','age', 'gender', 'boneage', 'radiologist', 'year_entry', 'path'])

In [12]:
mexico_data.dropna(inplace=True)
mexico_data['gender'] = mexico_data.gender.astype(int)

In [15]:
mexico_data

,id,age,gender,boneage,radiologist,year_entry,path
0,1,84,0,72,JSA,2023,"data/Mexico_private_dataset/2023\AGAMI KALACH,..."
1,2,112,0,84,JSA,2023,"data/Mexico_private_dataset/2023\AGAMI KALACH,..."
2,3,63,1,36,JSA,2023,"data/Mexico_private_dataset/2023\AGAMI KALACH,..."
3,4,135,0,132,JSA,2023,"data/Mexico_private_dataset/2023\AGAMI KALACH,..."
4,5,154,1,144,JSA,2023,data/Mexico_private_dataset/2023\AGUILAR ALVAR...
...,...,...,...,...,...,...,...
111,112,97,1,84,JSA,2023,data/Mexico_private_dataset/2023\SAADIA DICKTE...
112,113,76,1,66,JSA,2023,data/Mexico_private_dataset/2023\SAADIA DICKTE...
113,114,138,1,144,JSA,2023,"data/Mexico_private_dataset/2023\SACAL ROMANO,..."
114,115,111,0,84,JSA,2023,data/Mexico_private_dataset/2023\SAINZ GONZALE...


In [14]:
# save pandas df as csv without index
mexico_data.to_csv("./data/Mexico_private_dataset/mexico_additional_data.csv", index=False)

### Run the SAM model and continue from here

In [16]:
mexico_data

,id,age,gender,boneage,radiologist,year_entry,path
0,1,84,0,72,JSA,2023,"data/Mexico_private_dataset/2023\AGAMI KALACH,..."
1,2,112,0,84,JSA,2023,"data/Mexico_private_dataset/2023\AGAMI KALACH,..."
2,3,63,1,36,JSA,2023,"data/Mexico_private_dataset/2023\AGAMI KALACH,..."
3,4,135,0,132,JSA,2023,"data/Mexico_private_dataset/2023\AGAMI KALACH,..."
4,5,154,1,144,JSA,2023,data/Mexico_private_dataset/2023\AGUILAR ALVAR...
...,...,...,...,...,...,...,...
111,112,97,1,84,JSA,2023,data/Mexico_private_dataset/2023\SAADIA DICKTE...
112,113,76,1,66,JSA,2023,data/Mexico_private_dataset/2023\SAADIA DICKTE...
113,114,138,1,144,JSA,2023,"data/Mexico_private_dataset/2023\SACAL ROMANO,..."
114,115,111,0,84,JSA,2023,data/Mexico_private_dataset/2023\SAINZ GONZALE...


In [17]:
mexico_data['path'] = mexico_data['id'].map(lambda x: os.path.join("data",
                                                         "Mexico_private_dataset",
                                                         'additional',
                                                         '{}.png'.format(x)))

In [18]:
mexico_data

,id,age,gender,boneage,radiologist,year_entry,path
0,1,84,0,72,JSA,2023,data\Mexico_private_dataset\additional\1.png
1,2,112,0,84,JSA,2023,data\Mexico_private_dataset\additional\2.png
2,3,63,1,36,JSA,2023,data\Mexico_private_dataset\additional\3.png
3,4,135,0,132,JSA,2023,data\Mexico_private_dataset\additional\4.png
4,5,154,1,144,JSA,2023,data\Mexico_private_dataset\additional\5.png
...,...,...,...,...,...,...,...
111,112,97,1,84,JSA,2023,data\Mexico_private_dataset\additional\112.png
112,113,76,1,66,JSA,2023,data\Mexico_private_dataset\additional\113.png
113,114,138,1,144,JSA,2023,data\Mexico_private_dataset\additional\114.png
114,115,111,0,84,JSA,2023,data\Mexico_private_dataset\additional\115.png


In [19]:
mexico_data.gender.value_counts()

gender
1    61
0    55
Name: count, dtype: int64

In [20]:
# Checking if all the images exist
mexico_data['exists'] = mexico_data['path'].map(os.path.exists)
print(mexico_data['exists'].sum(), 'images found of', mexico_data.shape[0], 'total')

108 images found of 116 total


In [21]:
# Drop row if exist column is false does not exist
mexico_data = mexico_data[mexico_data['exists']]

In [22]:
mexico_data.to_csv("./data/Mexico_private_dataset/mexico_preprocessed_additional.csv", index=False)


In [3]:
# Train test split 5 fold on mexico_data
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

In [5]:
mexico_data = pd.read_csv("./data/Mexico_private_dataset/mexico_complete_dataset.csv")

In [6]:
from sklearn.model_selection import StratifiedKFold

# Separate the features from the target variable
X = mexico_data[['age', 'gender']]
y = mexico_data['boneage']

# Create an instance of StratifiedKFold with 5 folds
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Perform the stratified split and print the indices for each fold
fold = 1
splits = {}
for train_index, test_index in skf.split(X, y):
    splits[fold] = {}
    splits[fold]['train'] = train_index
    splits[fold]['val'] = test_index
    fold += 1

D:\Installation\anaconda\envs\bone_age_estimation\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


In [7]:
# Save dictionary
import pickle
with open("data/Mexico_private_dataset/splits_10.pkl", "wb") as f:
    pickle.dump(splits, f)

In [8]:
splits

{1: {'train': array([  0,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
          14,  15,  16,  18,  19,  20,  21,  22,  23,  24,  26,  27,  29,
          30,  31,  32,  33,  35,  36,  37,  38,  39,  40,  41,  42,  43,
          44,  46,  47,  48,  49,  50,  51,  52,  54,  55,  56,  57,  58,
          61,  64,  65,  66,  67,  68,  69,  70,  71,  73,  74,  75,  76,
          77,  78,  79,  80,  81,  82,  83,  84,  86,  87,  88,  89,  90,
          91,  92,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
         105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 117, 118,
         119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131,
         132, 133, 134, 135, 136, 137, 138, 139, 141, 143, 144, 145, 146,
         148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160,
         161, 162, 163, 164, 165, 167, 169, 170, 171, 173, 174, 175, 176,
         177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189,
         190, 191, 192, 19

## Mixing new private dataset

In [3]:
import pandas as pd
import numpy as np
import glob

In [26]:
# Load the new private dataset
new_private_data = pd.read_csv("./data/Mexico_private_dataset/mexico_preprocessed_additional.csv")
old_private_data = pd.read_csv("./data/Mexico_private_dataset/mexico_preprocessed_dataset.csv")

In [27]:
len(new_private_data), len(old_private_data)

(108, 341)

In [28]:
# Combine both pandas dataframes and reindex the id column

In [29]:
# Use the last id from the old private dataset
new_private_data['id'] = new_private_data['id'] + old_private_data['id'][len(old_private_data) - 1]

In [30]:
new_private_data

,id,age,gender,boneage,radiologist,year_entry,path,exists
0,346,84,0,72,JSA,2023,data\Mexico_private_dataset\additional\1.png,True
1,347,112,0,84,JSA,2023,data\Mexico_private_dataset\additional\2.png,True
2,349,135,0,132,JSA,2023,data\Mexico_private_dataset\additional\4.png,True
3,351,114,1,120,JSA,2023,data\Mexico_private_dataset\additional\6.png,True
4,352,160,1,180,JSA,2023,data\Mexico_private_dataset\additional\7.png,True
...,...,...,...,...,...,...,...,...
103,457,97,1,84,JSA,2023,data\Mexico_private_dataset\additional\112.png,True
104,458,76,1,66,JSA,2023,data\Mexico_private_dataset\additional\113.png,True
105,459,138,1,144,JSA,2023,data\Mexico_private_dataset\additional\114.png,True
106,460,111,0,84,JSA,2023,data\Mexico_private_dataset\additional\115.png,True


In [31]:
# Open the images from the path column
import cv2
for row in new_private_data.iterrows():
    image = cv2.imread(row[1]['path'], cv2.IMREAD_UNCHANGED)
    cv2.imwrite(f"./data/Mexico_private_dataset/additional/{row[1]['id']}.png", image)
    # delete the old image
    os.remove(row[1]['path'])
    # Rename the path value in the path column
    new_private_data.loc[row[0], 'path'] = f"./data/Mexico_private_dataset/additional/{row[1]['id']}.png"


In [32]:
# combine both dataframe
mexico_data = pd.concat([old_private_data, new_private_data])


In [33]:
len(mexico_data)

449

In [40]:
# save pandas df as csv without index
mexico_data.to_csv("./data/Mexico_private_dataset/mexico_complete_dataset.csv", index=False)